<a href="https://colab.research.google.com/github/dajebbar/FreeCodeCamp-python-data-analysis/blob/main/Exercise_M3_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

adult_census = pd.read_csv("./adult.csv")

target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=[target_name, "education-num"])

data_train, data_test, target_train, target_test = train_test_split(
    data, target, train_size=0.2, random_state=42)

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OrdinalEncoder

categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value",
                                          unknown_value=-1)
preprocessor = ColumnTransformer(
    [('cat-preprocessor', categorical_preprocessor,
      selector(dtype_include=object))],
    remainder='passthrough', sparse_threshold=0)


from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

model = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", HistGradientBoostingClassifier(random_state=42))
])

In [3]:
for param in model.get_params():
  print(param)

memory
steps
verbose
preprocessor
classifier
preprocessor__n_jobs
preprocessor__remainder
preprocessor__sparse_threshold
preprocessor__transformer_weights
preprocessor__transformers
preprocessor__verbose
preprocessor__verbose_feature_names_out
preprocessor__cat-preprocessor
preprocessor__cat-preprocessor__categories
preprocessor__cat-preprocessor__dtype
preprocessor__cat-preprocessor__handle_unknown
preprocessor__cat-preprocessor__unknown_value
classifier__categorical_features
classifier__early_stopping
classifier__l2_regularization
classifier__learning_rate
classifier__loss
classifier__max_bins
classifier__max_depth
classifier__max_iter
classifier__max_leaf_nodes
classifier__min_samples_leaf
classifier__monotonic_cst
classifier__n_iter_no_change
classifier__random_state
classifier__scoring
classifier__tol
classifier__validation_fraction
classifier__verbose
classifier__warm_start


In [13]:
from sklearn.model_selection import cross_val_score, KFold

lrs = [0.01, 0.1, 1, 10]
max_leafs = [3, 10, 30]
best_score = 0
best_params = {}

cv = KFold(n_splits=10, shuffle=True, random_state=42)

for lr in lrs:
  for node in max_leafs:
    print(f"Evaluating model with learning rate {lr:.3f}"
              f" and max leaf nodes {node}... ", end="")
    model.set_params(classifier__learning_rate=lr,
                    classifier__max_leaf_nodes=node )
  
    scores = cross_val_score(
      model,
      data_train,
      target_train,
      cv=cv,
      n_jobs=2
  )
    mean_score = scores.mean()
    print(f"score: {mean_score:.3f}")
    if mean_score > best_score:
            best_score = mean_score
            best_params = {'learning-rate': lr, 'max leaf nodes': node}
            print(f"Found new best model with score {best_score:.3f}!")

print(f"The best accuracy obtained is {best_score:.3f}")
print(f"The best parameters found are:\n {best_params}")


Evaluating model with learning rate 0.010 and max leaf nodes 3... score: 0.790
Found new best model with score 0.790!
Evaluating model with learning rate 0.010 and max leaf nodes 10... score: 0.813
Found new best model with score 0.813!
Evaluating model with learning rate 0.010 and max leaf nodes 30... score: 0.842
Found new best model with score 0.842!
Evaluating model with learning rate 0.100 and max leaf nodes 3... score: 0.850
Found new best model with score 0.850!
Evaluating model with learning rate 0.100 and max leaf nodes 10... score: 0.860
Found new best model with score 0.860!
Evaluating model with learning rate 0.100 and max leaf nodes 30... score: 0.859
Evaluating model with learning rate 1.000 and max leaf nodes 3... score: 0.856
Evaluating model with learning rate 1.000 and max leaf nodes 10... score: 0.836
Evaluating model with learning rate 1.000 and max leaf nodes 30... score: 0.804
Evaluating model with learning rate 10.000 and max leaf nodes 3... score: 0.288
Evaluati

In [14]:
best_lr = best_params['learning-rate']
best_mln = best_params['max leaf nodes']

model.set_params(classifier__learning_rate=best_lr,
                 classifier__max_leaf_nodes=best_mln)
model.fit(data_train, target_train)
test_score = model.score(data_test, target_test)

print(f"Test score after the parameter tuning: {test_score:.3f}")

Test score after the parameter tuning: 0.869
